In [1]:
import altair as alt
import polars as pl
import yfinance as yf


In [4]:
def fetch_intraday(stock_symbol, period = "5d", interval = "1m"):
    data = yf.download(tickers=stock_symbol, period=period, interval=interval)
    pl_data = pl.DataFrame(data.reset_index())
    return pl_data


stock_symbol = "META"
data = fetch_intraday(stock_symbol)

[*********************100%%**********************]  1 of 1 completed


In [5]:
data = data.with_columns([
	pl.col("Datetime").dt.strftime("%Y-%m-%d").alias("Date")  # extract the date for grouping
])

# calc 1-day relative volume by averaging each days volume and comparing it to the overall mean
day_grouped = data.group_by("Date").agg([
	pl.col("Volume").mean().alias("Day_Avg_Volume")  # avg volm/day
])

In [7]:
# join it with daily data back to the original dataset
data = data.join(day_grouped, on="Date")

# calc 1d & 5d relative volm
data = data.with_columns([
	(pl.col("Volume")/ pl.col("Day_Avg_Volume")).alias("1D_rel_volm"),
	(pl.col("Volume")/ pl.col("Volume").mean()).alias("5D_rel_volm")
])

# compute returns based on closing price
data = data.with_columns([
	((pl.col("Close") / pl.col("Close").shift(1)) - 1).alias("Return")
])

# remove any rows with missing data
data = data.drop_nulls()

In [8]:
# must convert polars dataframe to pandas to use with altair
pandas_df = data.to_pandas()

In [9]:
# now plot the scatter plot
scatter_plot_1d = alt.Chart(pandas_df).mark_circle(size=60).encode(
	x = '1D_rel_volm',
	y = 'Return',
	tooltip=["Datetime", "Close", "Volume", "1D_rel_volm", "Return"]
).properties(
	title = f"1-Day Returns vs Relative Volume for {stock_symbol}",
	width = 600,
	height = 400
)

scatter_plot_5d = alt.Chart(pandas_df).mark_circle(size=60).encode(
	x = '5D_rel_volm',
	y = 'Return',
	tooltip=["Datetime", "Close", "Volume", "5D_rel_volm", "Return"]
).properties(
	title = f"5-Day Returns vs Relative Volume for {stock_symbol}",
	width = 600,
	height = 400
)

In [11]:
scatter_plot_1d.display()
scatter_plot_5d.display()

alt.Chart(...)

alt.Chart(...)